**Навигация по уроку**
1. [Галлюцинации в LLM](https://colab.research.google.com/drive/19U6Hv3rpK7kOTCO2-HKYbdx65-Ku3te0)
2. [Knowledge Graph. Борьба с галлюцинациями](https://colab.research.google.com/drive/1jUSnsx-QrLK0fLmzjryAq-mPbfTPg4tr)
3. [Knowledge Graph в действии](https://colab.research.google.com/drive/1rRQYo0VYvbDG-wfm43bTDHC_u4XErPyW)
4. [Домашняя работа](https://colab.research.google.com/drive/1psJR6wk6i7MbtYvOu_pM2M6bsoReE5pD)

**В домашней работе вы должны сделать:**
1. **На 3 балла**. Собрать интерфейс Gradio для модели из урока. В качестве данных выбрать любую статью или статьи из википедии. Должна быть кнопка распарсить данные и отправить запрос.

2. **На 4 балла**. Изучите представленные [Ридеры](https://llamahub.ai/?tab=readers) на Llama Hub. С помощью выбранного Ридера (любой кроме википедии) загрузите данные в графовую базу данных. И выполните первое задание ДЗ с этим Ридером.

3. **На 5 баллов**. Выполните первое задание. В качестве модели используйте любую русскоязычную модель от [SberDevice](https://huggingface.co/ai-forever). Изучите порядок загрузки модели (он изменится), как поменяется промпт и настройки модели.



# Модель FRED-T5-1.7B

Особенностью модели является возможность использования префиксов `'<LM>', '<SC1>',.. '<SC6>'`

- **Вариант с `<LM>` (Language Modeling)**
```
prompt = f"<LM>Контекст: {graph_db_context}\nВопрос: {user_question}\nОтвет:"
```

**Особенности:**

Режим работы: Активирует "чистое" языковое моделирование, где модель генерирует текст максимально естественно, опираясь на контекст.

**Преимущества:**

Ответы более креативные и развернутые, так как <LM> не накладывает жестких ограничений.

Лучше подходит для свободных диалогов или генерации объяснений.

**Недостатки:**

Может игнорировать контекст, если он противоречит предобученным знаниям модели.

Риск галлюцинаций (вымышленных фактов).

**Пример ответа:**

"Иван Грозный был первым царем России, правившим с 1547 по 1584 год. Его правление ознаменовалось созданием стрелецкого войска и взятием Казани."

---


- **Вариант без префикса, но с явной инструкцией**

```
prompt = f"Ответь на вопрос, используя контекст:\nКонтекст: {graph_db_context}\nВопрос: {user_question}\nОтвет:"
```
**Особенности:**

Режим работы: Модель интерпретирует запрос как задачу точного QA, где нужно строго следовать контексту.

**Преимущества:**

Ответы более точные и консервативные.

Меньше риск галлюцинаций, так как модель явно ориентирована на предоставленные данные.

**Недостатки:**

Ответы могут быть сухими или короткими.

Если контекст неполный, модель не станет "додумывать" информацию.

**Пример ответа:**

"Согласно контексту, Иван Грозный стал царем в 1547 году."

---
Когда что использовать?

**`<LM>`-вариант, если:**

- Нужны развернутые объяснения (например, для образовательного ассистента).

- Контекст из графовой БД неполный, и вы хотите, чтобы модель "додумала" логичные детали.

**Вариант с явной инструкцией, если:**

- Требуется максимальная точность (например, в медицинском или юридическом ассистенте).

- Контекст из графовой БД полный и достоверный, и нужно избегать отклонений.

**Префиксы `<SC1>-<SC6>`** в модели FRED-T5-1.7B предназначены для задач Span Corruption — это разновидность денойзинга (восстановления замаскированных фрагментов текста), где модель учится предсказывать пропущенные части входных данных.

Каждый префикс `<SCi>` соответствует разным параметрам маскирования:

µ (mu) — средняя длина пропуска (в токенах).

r (ratio) — доля замаскированных токенов в тексте.


| Префикс | µ (средняя длина пропуска) | r (доля маскирования) | Когда использовать                     |
|---------|----------------------------|-----------------------|----------------------------------------|
| `<SC1>` | 3 токена                   | 15%                   | Короткие пропуски, мало шума           |
| `<SC2>` | 8 токенов                  | 15%                   | Средние пропуски, мало шума            |
| `<SC3>` | 64 токена                  | 15%                   | Длинные пропуски, мало шума            |
| `<SC4>` | 3 токена                   | 50%                   | Короткие пропуски, сильный шум         |
| `<SC5>` | 8 токенов                  | 50%                   | Средние пропуски, сильный шум          |
| `<SC6>` | 64 токена                  | 50%                   | Длинные пропуски, сильный шум          |

Примеры использования `<SCi>` префиксов
1. **Короткие пропуски** (`SC1>, <SC4>`)
Подходит для восстановления отдельных слов или коротких фраз.

Пример с `<SC1>` (µ=3, r=15%):

```
prompt = "<SC1>Москва — столица <extra_id_0>. Население города составляет около 12 миллионов человек."
# Модель может предсказать: "<extra_id_0> России</s>"
Пример с <SC4> (µ=3, r=50%) — больше замаскированных токенов:
```
```
prompt = "<SC4><extra_id_0> — столица <extra_id_1>. Население <extra_id_2> составляет около 12 миллионов человек."
# Возможный вывод: "<extra_id_0> Москва <extra_id_1> России <extra_id_2> города</s>"
```
2. **Средние пропуски** (`<SC2>, <SC5>`)
Подходит для восстановления предложений или небольших абзацев.

Пример с `<SC2>` (µ=8, r=15%):

```
prompt = "<SC2>Вторая мировая война началась в <extra_id_0>. Основными участниками были Германия, СССР, США и Великобритания."
# Модель может предсказать: "<extra_id_0> 1939 году</s>"
```
Пример с `<SC5>` (µ=8, r=50%) — сильное маскирование:

```
prompt = "<SC5><extra_id_0> началась в <extra_id_1>. <extra_id_2> участниками были <extra_id_3>."
# Возможный вывод: "<extra_id_0> Вторая мировая война <extra_id_1> 1939 году <extra_id_2> Основными <extra_id_3> Германия, СССР, США и Великобритания</s>"
```
3. **Длинные пропуски** (`<SC3>, <SC6>`)
Подходит для восстановления целых абзацев или сложных структур.

Пример с `<SC3>` (µ=64, r=15%):

```
prompt = "<SC3>Французская революция <extra_id_0> привела к отмене монархии."
# Модель может восстановить целый абзац: "<extra_id_0>, начавшаяся в 1789 году и продолжавшаяся до 1799 года,</s>"
```
Пример с `<SC6>` (µ=64, r=50%) — экстремальное маскирование:

```
prompt = "<SC6><extra_id_0> <extra_id_1> привела к <extra_id_2>."
# Возможный вывод: "<extra_id_0> Французская революция <extra_id_1>, длившаяся с 1789 по 1799 год, <extra_id_2> отмене монархии и установлению республики</s>"
```

Возвращаемые запросы и ответы содержат `extra_id_i`, который указывает место в тексте, которое модели необходимо заполнить своим ответом.

# Установка зависимостей

In [1]:
!pip install --upgrade \
    numpy==1.26.0 \
    packaging==24.2 \
    requests==2.32.3 \
    tqdm==4.67.1 \
    filelock==3.15.1 \
    typing-extensions==4.14.0 \
    torch==2.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install \
    transformers==4.42.0 \
    llama-index==0.10.46 \
    pyvis==0.3.2 \
    langchain==0.2.5 \
    pypdf==4.2.0 \
    langchain-community==0.2.5 \
    llama-index-llms-huggingface==0.2.3 \
    llama-index-embeddings-huggingface==0.2.2 \
    llama-index-embeddings-langchain==0.1.2 \
    langchain-huggingface==0.0.3 \
    sentencepiece==0.1.99 \
    accelerate==0.31.0 \
    bitsandbytes==0.43.1 \
    peft==0.11.1 \
    llama-index-readers-wikipedia==0.1.4 \
    wikipedia==1.4.0 \
    safetensors==0.4.3 \
    tokenizers==0.19.1 \
    huggingface-hub==0.23.3 \
    gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-program-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking a

## Импорт

In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import PromptTemplate
from llama_index.embeddings.langchain import LangchainEmbedding
from transformers import GPT2Tokenizer, T5ForConditionalGeneration
from peft.peft_model import PeftModel
from peft.config import PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from pyvis.network import Network
from huggingface_hub import login
from google.colab import userdata

In [2]:
HF_TOKEN=userdata.get('HF_TOKEN')
# Вставьте ваш токен (здесь указан временный токен)
login(HF_TOKEN, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Настройка модели

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device='cuda'
model.to(device)

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.96G [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

## Проверка работы

In [ ]:
#Prefix <LM>
lm_text='<LM>Принялся Кутузов рассказывать свою историю как он сюда попал. Началось'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs=model.generate(input_ids, eos_token_id=tokenizer.eos_token_id, early_stopping=True)
print(tokenizer.decode(outputs[0][1:]))

# print result: с того, что он был в армии, служил в артиллерии</s>.

#Prefix <SC1>
lm_text='<SC1>Принялся Кутузов рассказывать свою историю <extra_id_0>. Началось с того, что он был в армии, служил в артиллерии.'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True)
print(tokenizer.decode(outputs[0][1:]))

#print result: '<extra_id_0>, как он воевал</s>'

# Prefix <SC5>
lm_text='<SC5>Принялся Кутузов рассказывать свою историю <extra_id_0>. Началось с того, что он был в армии, служил в артиллерии.'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True)
tokenizer.decode(outputs[0][1:])

#print result: '<extra_id_0>, как он стал генералом</s>'


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 с того, что он был в армии, служил в артиллерии.</s>
<extra_id_0>  и рассказывать интересно</s>


'<extra_id_0> \n— Вот как было дело</s>'

In [ ]:
lm_text='<LM>Задание: Дай точное определение термина.\nТермин: компьютерные нейросети\nОпределение:'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True, max_new_tokens=200)
print(tokenizer.decode(outputs[0][1:]))

 компьютерные нейросети - это системы, которые могут обучаться и самообучаться.</s>


In [ ]:
#Prefix <SC1>
lm_text='<SC1>Дай определение: Нейросети - <extra_id_0>'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True, max_new_tokens=100)
print(tokenizer.decode(outputs[0][1:]))


<extra_id_0>  Искусственный интеллект.</s>


In [ ]:
# Prefix <SC5>
lm_text='<SC5>Дай развернутое определение: Нейросеть - <extra_id_0>'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True, max_new_tokens=100)
tokenizer.decode(outputs[0][1:])

'<extra_id_0>  Искусственный интеллект, который может выполнять сложные задачи.</s>'

In [ ]:
# Prefix <LM>
lm_text='<LM>Столица Франции - <extra_id_0>, а Германии - <extra_id_1>. Обе находятся в <extra_id_2>'
input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True, max_new_tokens=100)
tokenizer.decode(outputs[0][1:])

'<extra_id_0> \nПариж <extra_id_1> \nБерлин <extra_id_2> \nЕвропе.</s>'

In [ ]:
def format_definition_request(term):
    return f"""<LM>Система: Ты - эксперт по терминологии. Дай строгое определение.
Пользователь: Определи термин "{term}"
Ассистент: {term} - это"""

# Использование
prompt = format_definition_request("Нейросети")
input_ids=torch.tensor([tokenizer.encode(prompt)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,early_stopping=True, max_new_tokens=100)
tokenizer.decode(outputs[0][1:])

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


' нейронные сети, которые могут обучаться.</s>'

Для генерации ответов на натуральные вопросы с поиском по документам в FRED-T5-1.7B оптимально использовать префикс `<LM>` (Language Modeling), а не `<SC>`-префиксы, которые требуют явного указания пропусков (что не подходит для свободных вопросов пользователя).

## Форматирование для FRED-T5

In [4]:
def messages_to_prompt(messages):
    """Адаптация формата <LM> для диалога"""
    prompt = "<LM>"
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>System: {message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>User: {message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<s>Assistant: {message.content}</s>\n"
    return prompt + "<s>Assistant:\n"

def completion_to_prompt(completion):
    """Форматирование одиночных запросов"""
    return f"<LM><s>System:</s>\n<s>User: {completion}</s>\n<s>Assistant:</s>\n<s>Assistant:"

## Инициализация LLM в LlamaIndex

In [5]:
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    model_name='ai-forever/FRED-T5-1.7B',
    #context_window=2048,
    max_new_tokens=512,
    generate_kwargs={
        "early_stopping": True,
        "temperature": 0.3,
        "top_k": 30,
        "top_p": 0.9,
        "repetition_penalty": 1.1,
        "do_sample": True,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.pad_token_id,
    },
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="cuda",
)

In [ ]:
response = llm.complete("Задание: Расскажи про Кутузова для доклада", reset_context=True)
print(response.text)  # вывод сгенерированного текста

 1813 году Кутузов был назначен главнокомандующим. В этом же году он участвовал в сражении при Бородино.
Кутузов был одним из самых выдающихся полководцев Отечественной войны 1812 года. Он командовал армией во время сражения под Москвой.
Память о нем сохранилась в виде памятника на Бородинском поле.


In [ ]:
prompt = completion_to_prompt("расскажи про Кутузова")
print(prompt)

<LM><s>System:</s>
<s>User: расскажи про Кутузова</s>
<s>Assistant:</s>
<s>Assistant:


## Настройки окружения LlamaIndex

In [ ]:
from langchain_huggingface  import HuggingFaceEmbeddings
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                        model_kwargs={"device": "cuda"},
                        encode_kwargs={"batch_size": 32}
                        )
)

In [ ]:
# Настройка ServiceContext (глобальная настройка параметров LLM)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [10]:
# Создаем простое графовое хранилище
graph_store = SimpleGraphStore()

# Устанавливаем информацию о хранилище в StorageContext
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# Проблемы с моделью и решения

Загрузить квантованную модель не получилось из-за конфликта библиотеки BytesandBitsConfig и Triton, однако, как оказалось, полная модель смогла запуститься без проблем на бесплатно предоставляемом железе colab. По этому использовал метод, указанный в инструкции на HF.

При попытке несколько раз выполнить генерацию ответа, модель постепенно начинала галлюцинировать, причина - переполнение кешированного контекста HuggingFaceLLM. Помог параметр `reset_context=True` в запросе `llm.complete()`

Долго пытался подобрать правильное форматирование запроса для модели - верный формат: `<LM>` тег должен начинать весь промпт, включая `System` и `Assistant`, а не только текст пользователя. Тем не менее артефакты в генерации встречаются - начало

## Добавление документов pdf в базу

Работать будем с простыми книгами в формате pdf - по этому нет необходимости в сложных ридерах. PyMuPDF простой и быстрый ридер - подойдет для таких целей.

In [5]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 83.9 MB/s eta 0:00:00


# Интерфейс и сборка

In [6]:
!pip install --upgrade gradio==3.50.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.3.0
    Uninstalling gradio_client-1.3.0:
      Successfully uninstalled gradio_client-1.3.0
  Attempting uninstall: gradio
    Found existing installation: gradio 4.44.1
    Uninstalling gradio-4.44.1:
      Successfully uninstalled gradio-4.44.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.23.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 11.

In [10]:
import gradio as gr
import os
import traceback
import time
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.query_engine import RetrieverQueryEngine
import tempfile
from langchain_huggingface  import HuggingFaceEmbeddings

torch.cuda.empty_cache()  # Очищаем кэш GPU
torch.backends.cudnn.benchmark = True  # Оптимизация для CUDA

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        model_kwargs={
            "device": "cuda",
            "trust_remote_code": True
        },
        encode_kwargs={
            "batch_size": 64,  # Увеличиваем batch size
            "normalize_embeddings": True,  # Включаем нормализацию
            "show_progress_bar": True
        }
    )
)

# Глобальные переменные
current_index = None
DEFAULT_SYSTEM_PROMPT = "Ты помощник обработки статей, отвечай только основываясь на данных из статьи. Если не знаешь ответ - напиши 'нет данных', не придумывай."

# Доработанные функции промптов
def messages_to_prompt(messages, system_prompt=DEFAULT_SYSTEM_PROMPT):
    prompt = "<LM>"
    if system_prompt:
        prompt += f"<s>System: {system_prompt}</s>\n"
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>System: {message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>User: {message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<s>Assistant: {message.content}</s>\n"
    return prompt + "<s>Assistant:\n"

def completion_to_prompt(completion, system_prompt=DEFAULT_SYSTEM_PROMPT):
    base_prompt = "<LM>"
    if system_prompt:
        base_prompt += f"<s>System: {system_prompt}</s>\n"
    else:
        base_prompt += "<s>System:</s>\n"
    return f"{base_prompt}<s>User: {completion}</s>\n<s>Assistant:</s>\n<s>Assistant:"

llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    model_name='ai-forever/FRED-T5-1.7B',
    #context_window=2048,
    max_new_tokens=512,
    generate_kwargs={
        "early_stopping": True,
        "temperature": 0.3,
        "top_k": 30,
        "top_p": 0.9,
        "repetition_penalty": 1.1,
        "do_sample": True,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.pad_token_id,
    },
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="cuda",
)

# Настройка ServiceContext (глобальная настройка параметров LLM)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 2048
Settings.chunk_overlap = 200
Settings.num_output=512

def parse_pdf(file_path):
    global current_index

    try:
        start_time = time.time()
        actual_path = file_path.name

        if not os.path.exists(actual_path) or os.path.getsize(actual_path) == 0:
            return "❌ Ошибка: файл не загружен или пустой"

        print(f"Начинаем обработку файла: {os.path.getsize(actual_path)} байт")

        # Загрузка PDF
        loader = PyMuPDFReader()
        documents = loader.load(actual_path)

        print(f"Загружено {len(documents)} документов")

        # Проверяем содержимое документов
        for i, doc in enumerate(documents[:2]):  # Показываем первые 2 документа
            print(f"Документ {i}: {doc.text[:200]}...")

        # Очищаем настройки и устанавливаем заново
        Settings.embed_model = embed_model
        Settings.llm = llm

        # Создание графовой базы с явным указанием embed_model
        graph_store = SimpleGraphStore()
        storage_context = StorageContext.from_defaults(graph_store=graph_store)

        print("Создаем Knowledge Graph Index...")
        current_index = KnowledgeGraphIndex.from_documents(
            documents,
            storage_context=storage_context,
            max_triplets_per_chunk=2,
            include_embeddings=True,
            embed_model=embed_model,  # Явно передаем embed_model
            show_progress=True
        )

        # Принудительно устанавливаем embed_model в созданный индекс
        current_index._embed_model = embed_model

        elapsed_time = time.time() - start_time
        return f"✅ PDF успешно обработан за {elapsed_time:.2f} секунд. Графовая база создана."

    except Exception as e:
        error_trace = traceback.format_exc()
        return f"❌ Ошибка:\n{str(e)}\n\nПодробности:\n{error_trace}"


def query_index(user_query):
    global current_index
    if not current_index:
        return "⚠ Сначала загрузите и обработайте PDF файл"

    try:
        print(f"Обрабатываем запрос: {user_query}")

        # Альтернативный подход - используем retriever напрямую
        from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

        # Создаем retriever с принудительным использованием эмбеддингов
        retriever = KnowledgeGraphRAGRetriever(
            storage_context=current_index._storage_context,
            service_context=None,
            llm=llm,
            embed_model=embed_model,  # Явно передаем embed_model
            verbose=True,
            similarity_top_k=5,
            graph_traversal_depth=2,
            max_knowledge_sequence=512
        )

        # Создаем query engine с retriever
        from llama_index.core.query_engine import RetrieverQueryEngine
        query_engine = RetrieverQueryEngine.from_args(
            retriever=retriever,
            service_context=None,
            response_mode="compact",
            verbose=True
        )

        print("Query engine создан, выполняем запрос...")
        response = query_engine.query(user_query)

        print(f"Получен ответ: {response}")

        if response is None or str(response).strip() == "" or str(response) == "Empty Response":
            # Fallback - пробуем простой поиск по тексту
            print("Пробуем fallback поиск...")
            return fallback_search(user_query)

        return str(response)

    except Exception as e:
        error_trace = traceback.format_exc()
        print(f"Ошибка при обработке запроса: {error_trace}")
        # Пробуем fallback
        return fallback_search(user_query)

def fallback_search_with_llm(user_query):
    """Fallback с использованием настроенного HuggingFaceLLM и функций промптов"""
    global current_index, llm
    try:
        # Получаем релевантный контекст
        docstore = current_index._storage_context.docstore
        all_docs = docstore.docs

        query_lower = user_query.lower()
        relevant_texts = []

        for doc_id, doc in all_docs.items():
            text = doc.text
            # Ищем релевантные абзацы
            paragraphs = text.split('\n')
            for paragraph in paragraphs:
                if paragraph.strip() and any(word in paragraph.lower() for word in query_lower.split()):
                    relevant_texts.append(paragraph.strip())
                    if len(relevant_texts) >= 3:
                        break

        if relevant_texts:
            context = "\n".join(relevant_texts[:3])

            # Формируем системный промпт для контекстного ответа
            context_system_prompt = f"{DEFAULT_SYSTEM_PROMPT} Используй следующий контекст из документа для ответа: {context}"

            # Создаем промпт используя функцию completion_to_prompt
            formatted_prompt = completion_to_prompt(
                completion=user_query,
                system_prompt=context_system_prompt
            )

            print(f"Сформированный промпт: {formatted_prompt[:200]}...")

            # Используем настроенный LLM
            try:
                response = llm.complete(formatted_prompt)
                answer = str(response).strip()

                # Проверяем качество ответа
                if answer and answer != "Empty Response" and len(answer) > 10:
                    return answer
                else:
                    # Если LLM ответ пустой, возвращаем обработанный контекст
                    return extract_answer_from_context(context, user_query)

            except Exception as llm_error:
                print(f"Ошибка LLM: {llm_error}")
                # Fallback к простой обработке текста
                return extract_answer_from_context(context, user_query)
        else:
            return "⚠️ Информация по запросу не найдена в документе"

    except Exception as e:
        return f"❌ Ошибка fallback поиска: {str(e)}"

def extract_answer_from_context(context, query):
    """Улучшенная экстракция ответа из контекста"""
    lines = [line.strip() for line in context.split('\n') if line.strip()]

    # Ищем строки с ключевыми словами из вопроса
    query_words = [word.lower() for word in query.split() if len(word) > 2]

    # Ищем прямые ответы (пары ключ-значение)
    for line in lines:
        line_lower = line.lower()
        if any(word in line_lower for word in query_words):
            # Проверяем, есть ли в строке структурированная информация
            if any(sep in line for sep in [':', '\t']) and len(line.split()) >= 2:
                return f"Согласно документу: {line}"

    # Ищем релевантные предложения
    relevant_sentences = []
    for line in lines:
        if any(word in line.lower() for word in query_words):
            relevant_sentences.append(line)

    if relevant_sentences:
        # Объединяем первые 2 релевантных предложения
        answer_text = ". ".join(relevant_sentences[:2])
        return f"Согласно документу: {answer_text}"

    # Если ничего конкретного не найдено, возвращаем общий контекст
    return f"Найденная информация: {context[:300]}..."

# Интерфейс Gradio
with gr.Blocks() as demo:
    gr.Markdown("## PDF Knowledge Graph QA System")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Загрузите PDF файл", file_types=[".pdf"])
            parse_btn = gr.Button("Распарсить PDF")
            parse_output = gr.Textbox(label="Статус обработки")

        with gr.Column():
            gr.Markdown(f"**Системный промпт:** {DEFAULT_SYSTEM_PROMPT}")
            user_query = gr.Textbox(label="Ваш вопрос")
            query_btn = gr.Button("Задать вопрос")
            answer_output = gr.Textbox(label="Ответ системы", lines=10)

    parse_btn.click(parse_pdf, inputs=[file_input], outputs=parse_output)
    query_btn.click(query_index, inputs=[user_query], outputs=answer_output)

# Запускаем с настройками для Colab
if __name__ == "__main__":
    demo.queue().launch(
        debug=True,
        share=True,
        server_name="0.0.0.0",
        server_port=7870,
    )

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://4d5335b6de0ded9574.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Начинаем обработку файла: 841417 байт
Загружено 6 документов
Документ 0: Долина Царей
Общий вид на Долину Царей
Страна
Египет
Регион
Долина Царей
Координаты
25°44′25″ с. ш.
32°36′08″ в. д.
Дата основания
XVIII династия
Первое
упоминание
XVI век до н. э.
 Медиафайлы на Вики...
Документ 1: собой прямой коридор, по оси которого высекались колонные залы, сокровищницы, шахта и погребальная камера, и достаточно
глубоко уходят в толщу скалы. Третий тип, относящийся к XX династии (Рамсес III ...
Создаем Knowledge Graph Index...


Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


...

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Обрабатываем запрос: где находится Долина Царей?
Query engine создан, выполняем запрос...
Получен ответ: Empty Response
Пробуем fallback поиск...
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7870 <> https://4d5335b6de0ded9574.gradio.live


# Итог

Не сказать, что результат отличный, тем не менее цель выполнена. Использовали модель `ai-forever/FRED-T5-1.7B` от СберДевайс, создали интерфейс, загрузили файл pdf (исползовал pdf версию  случайной страницы wiki [Долина Царей](https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D0%BB%D0%B8%D0%BD%D0%B0_%D0%A6%D0%B0%D1%80%D0%B5%D0%B9), а не wiki-ридер, что позволяет отойти от самой вики. Скорость создания базы оказалась не высокой, документы 1.5 мб обрабатывались по 15 минут. Ответ от модели не сказать, чтобы развернутый, но начало вполне адекватное. Для дальнейшего улучшения стоило бы добавить кеширование базы.